# Data Analytics and Algorithms CA

## Importing all the required librabries and the dataset

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
from matplotlib import pyplot 
%matplotlib inline
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import mean_squared_error
series = read_csv('../Learning/data/Assignment Sample.csv', header=0, index_col=0)
df = pd.read_csv("../Learning/data/Assignment Sample.csv",parse_dates = [1], squeeze = True)
data1 = series

In [ ]:
import altair as alt
from vega_datasets import data
from sklearn.metrics import r2_score


## Introducing two new columns of summarized data from monsoon and non-monsson months

In [ ]:
data1['NONS'] = data1[['JAN','FEB','MAR','APR','MAY','NOV','DEC']].sum(axis=1)
data1['MONS'] = data1[['JUN','JUL','AUG','SEP','OCT']].sum(axis=1)
df['NONS'] = df[['JAN','FEB','MAR','APR','MAY','NOV','DEC']].sum(axis=1)
df['MONS'] = df[['JUN','JUL','AUG','SEP','OCT']].sum(axis=1)

## Getting the list of the different unique regions that are present in the dataset

In [ ]:
Region = set(list(data1.index))

===================================================================================================================

===================================================================================================================

# ARIMA model

### ARIMA (Integrated Autoregressive Moving Average) is a time series technique used for forecasting. In the forecasting equation, lags of the stationary series are called "autoregressive" terms, lags of the forecast errors are called "moving average" terms, and a time series that needs to be separated to be stationary is said to be an "integrated" variant of a stationary series.

### An ARIMA model has main three inputs:
###   • p is the number of autoregressive terms (by inspecting the Partial Autocorrelation (PACF) plot),
###   • d is the number of nonseasonal differences needed for stationarity, and
###   • q is the number of lagged forecast errors in the prediction equation (by inspecting the Partial Autocorrelation (PACF) plot).



In [ ]:
df_sample = df[df["SUBDIVISION"] == "Kerala"]

df_sample1 = df_sample[["YEAR","NONS"]]
df_sample1 = df_sample1.set_index("YEAR")

# Getting the moving mean

sample1_mean = df_sample1.rolling(window = 20).mean()

df_sample1.plot() 
sample1_mean.plot()

from sklearn.metrics import mean_squared_error

values = pd.DataFrame(df_sample1.values)

data_df = pd.concat([values,values.shift(1)], axis = 1)

data_df.columns = ["Actual","Forecast"]

data_df = data_df[1:]

# Calculating the mean square value

data_error = mean_squared_error(data_df.Actual,data_df.Forecast)

np.sqrt(data_error)

# Plotting the ACF and PACF plots

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

plot_acf(df_sample1)

plot_pacf(df_sample1)

# Implementing the ARIMA model

from  statsmodels.tsa.arima_model import ARIMA

# Preparing the train and test data

data_train = df_sample1[0:100]
data_test = df_sample1[100:117]

# Giving the p, d and q parameters for the ARIMA model

data_model = ARIMA(data_train, order = (2,1,2))

data_model_fit = data_model.fit()


data_forecast = data_model_fit.forecast(steps = 17)[0]
data_forecast_df = pd.DataFrame(data_forecast)
data_forecast_df["YEAR"] = ["2001-01-01","2002-01-01","2003-01-01","2004-01-01","2005-01-01",
                "2006-01-01","2007-01-01","2008-01-01","2009-01-01","2010-01-01",
                "2011-01-01","2012-01-01","2013-01-01","2014-01-01","2015-01-01",
                "2016-01-01","2017-01-01"]
data_forecast_df.columns = ["NONS","YEAR"]

data_forecast_df = data_forecast_df[["YEAR","NONS"]]

# Printing the forecast values 

print(data_forecast_df)

# Printing the values from the test samples

print(data_test)

data_test_df = pd.DataFrame(data_test)
error = mean_squared_error(data_test,data_forecast)

np.sqrt(error)

### For the above dataset, ARIMA is not an ideal forecasting model. The dataset has 36 regions, which have to be individually passed into the model for forecasting as they are having a unique set of (p, d, q) values that needs to be manually inserted by observing the PACF (Partial Autocorrelation) and ACF (Autocorrelation) plots.

## Visualization of the original dataset when they are divided into train and test datasets

In [ ]:
sample_values = df_sample1.values
train, test = sample_values[:len(sample_values)-17], sample_values[len(sample_values)-17:]

pyplot.plot(train)
pyplot.plot([None for i in train] + [x for x in test])
pyplot.show()

===================================================================================================================

===================================================================================================================

# Autoregression Model

### An autoregressive (AR) model predicts future behavior based on past behavior. It’s used for forecasting when there is some correlation between values in a time series and the values that precede and succeed them. 

### In an AR model, the value of the outcome variable (Y) at some point t in time is directly related to the predictor variable (X) and previous values for Y. This is the main difference between simple linear regression and AR models.

### The statsmodels library provides an autoregression model that automatically selects an appropriate lag value using statistical tests and trains a model. It is provided in the AR class. 



In [ ]:
def prediction(r, sample1):
    X = sample1.values
    
    # Preparing the train and test data
    
    train, test = X[1:len(X)-17], X[len(X)-17:]
    
    # train autoregression
    
    model = AR(train)
    model_fit = model.fit()
    window = model_fit.k_ar
    coef = model_fit.params
    mean = sample1.mean(axis = 0)
    
    # walk forward over time steps in test
    
    history = train[len(train)-window:]
    history = [history[i] for i in range(len(history))]
    predictions = list()
    for t in range(len(test)):
        length = len(history)
        lag = [history[i] for i in range(length-window,length)]
        yhat = coef[0]
        for d in range(window):
            yhat += coef[d+1] * lag[window-d-1]
        obs = test[t]
        predictions.append(yhat)
        history.append(obs)
        
        # Printing the predicted and the original values
        
        print('predicted=%f, expected=%f' % (yhat, obs))
    error = mean_squared_error(test, predictions)
    print('Test MSE: %.3f' % error)
    print(window)
    
    # plot
    
    pyplot.plot(test)
    pyplot.plot(predictions, color='red')
    pyplot.show()
    rms = r2_score(test,predictions)
    



In [ ]:
# Preparing a 'for' loop so that the above function for all the Regions in the dataset

for r in Region:
    a = data1.loc[r]
    b = df[df["SUBDIVISION"] == r]
    sample = pd.DataFrame(a)
    sample = sample[["YEAR","NONS"]]
    sample1 = sample.set_index('YEAR')
    print(r)
    
    # Calling the above function
    
    prediction(r, sample1)
   
    

### As the AR model is univariant, we summarized all the non-monsoon months for the predictor variable (X). This helps in predicting only the summarized value but not the individual month data. Also, this model performs quite well for some of the regions (as they are having stationary data) but does not fit for a few of the regions.

===================================================================================================================

===================================================================================================================

# VAR Model

### Vector Autoregression (VAR) is a multivariate forecasting algorithm that is used when two or more time-series influence each other. It is considered as an Autoregressive model because, each variable (Time Series) is modeled as a function of the past values, that is the predictors are nothing but the lags (time-delayed value) of the series.

### The primary difference is that the above models (both ARIMA and AR) are unidirectional, where, the predictors influence the Y and not vice-versa. Whereas, Vector Auto Regression (VAR) is bi-directional. Each variable in this model is modeled as a linear combination of past values of itself and the past values of other variables in the system. Thus, the variables influence each other.

### It can also be used for analysis of non-stationary multivariate time series 


In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
import matplotlib.pylab as plt1

In [ ]:
# Preparing a VAR model to train with the dataset

def var_model1(r1, train1, test1):
    yhat1 = [[]]
    model = VAR(sample21)
    test = test1
    Region = r1
    model_fit = model.fit()
    yhat = model_fit.forecast(model_fit.y, steps=17)
    df1 = pd.DataFrame(yhat)
    df1.columns = ["JAN","FEB","MAR","APR","MAY","NOV","DEC"]
    df1["YEAR"] = range(2001,2018)
    df1 = df1.set_index("YEAR")
    plot1 = df1["NOV"].values
    plot2 = test["NOV"].values
    
    # Displaying the predicted values from the VAR model
    
    print(df1)
    
    # Plotting the predicted values against the original test values
    
    pyplot.plot(plot1, color = 'red')
    pyplot.plot(plot2)
    pyplot.show()
     

    


In [ ]:
# Preparing a 'for' loop so that the above function for all the Regions in the dataset

for r1 in Region:
    a1 = data1.loc[r1]
    b1 = df[df["SUBDIVISION"] == r1]
    sample1 = pd.DataFrame(a1)
    sample1 = sample1[["YEAR","JAN","FEB","MAR","APR","MAY","NOV","DEC"]]
    sample21 = sample1.set_index('YEAR')
    
    # Preparing the train and test datasets
    
    train1, test1 = sample21[1:len(sample21)-17], sample21[len(sample21)-17:]
    print("*"*200)
    print(r1)
    
    # Calling the above prepared VAR model function and passing the necessary parameters
    
    var_model1(r1, train1, test1)
    
    

### Using the VAR model to predict the next 5 years of rainfall for the NonMonsoon months

In [ ]:
# Prepared a function to predict the future values using the same VAR model

def var_model(r, sample2):
    yhat1 = [[]]
    
    # Applying the in-built VAR model to the dataset
    
    model = VAR(sample2)
    Region = r
    model_fit = model.fit()
    yhat = model_fit.forecast(model_fit.y, steps=6)
    
    # Converting the above predicted values into a DataFrame
    
    df1 = pd.DataFrame(yhat)
    df1.columns = ["JAN","FEB","MAR","APR","MAY","NOV","DEC"]
    df1["YEAR"] = range(2018,2024)
    
    # Plotting the above converted DataFrame 
    
    df_Plot = df1.plot(x = "YEAR", y = ["JAN","FEB","MAR","APR","MAY","NOV","DEC"], kind = "barh", figsize=(30, 15), fontsize = 25)
    df_Plot.set_xlabel('Amount of Rainfall ', fontsize = 25)
    df_Plot.set_ylabel('Year', fontsize = 25)
    df_Plot.set_title(Region, fontsize = 25)
    df1 = df1.set_index('YEAR')
    
    # Printing the predicted values
    
    print(df1)
    return yhat
    




In [ ]:
# Preparing a 'for' loop to call the above function for the future prediction for each Region

for r in Region:
    a = data1.loc[r]
    b = df[df["SUBDIVISION"] == r]
    sample = pd.DataFrame(a)
    sample = sample[["YEAR","JAN","FEB","MAR","APR","MAY","NOV","DEC"]]
    sample2 = sample.set_index('YEAR')
    print("*"*220)
    print(r)
    
    # Calling the above function and passing the necessary parameters
    
    var_model(r, sample2)
    

### Since the research problem is about predicting the rainfall for each of the non-monsoon months, thus making multiple input data. This model allows multiple inputs to be taken and lets us estimate future values for each input.

In [ ]:
df[df["SUBDIVISION"] == r].tail(10)

## Preparing the data for backtesting

In [ ]:
# Preparing a DataFrame for the last Region which was used for prediction in the above function

a = pd.DataFrame(var_model(r, sample2))
a.columns = ["JAN","FEB","MAR","APR","MAY","NOV","DEC"]

In [ ]:
a

In [ ]:
# Getting the original data for the same Region

testing = df[df["SUBDIVISION"] == r]

testing = testing[["JAN","FEB","MAR","APR","MAY","NOV","DEC"]]

In [ ]:
testing

In [ ]:
# Concatenating the above prepared DataFrames

new = pd.concat([testing, a], axis=0)

new1 = new.reset_index()

new1 = new1.iloc[:,1:]

new1

new1['NONS'] = new1[['JAN','FEB','MAR','APR','MAY','NOV','DEC']].sum(axis=1)

new1 = new1["NONS"]

## Multiple Train-Test Splits Backtesting

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X = new1.values

# Providing the split input (which is used to split the dataset)

splits = TimeSeriesSplit(n_splits=2)
pyplot.figure(1)
index = 1
for train_index, test_index in splits.split(X):
    train = X[train_index]
    test = X[test_index]
    
    # Printing the number of values in each train and test sets
    
    print('Observations: %d' % (len(train) + len(test)))
    print('Training Observations: %d' % (len(train)))
    print('Testing Observations: %d' % (len(test)))
    
    # Plotting the train and test datasets
    
    pyplot.subplot(310 + index)
    pyplot.plot(train)
    pyplot.plot([None for i in train] + [x for x in test])
    index += 1
pyplot.show()

# Answer to the Research Question

## Research Question: Will the nonseasonal rainfall in India increase/decrease in the next 3 years?

## Answer:

### From a total of 36 different regions in India, 7 of the regions are expected to have high rainfall in May and November apart from the monsoon months (which extends from June to October every year).
### While high rainfall is expected only in May in 9 regions and in December for 3 of the regions.
### For the rest of the 17 regions, there is no high rainfall expected in the non-monsoon months for the next 3 years.